## Necessary imports

In [34]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from  tensorflow.keras.models import Sequential
from  tensorflow.keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random

## Instantiating

In [35]:
lemmatizer = WordNetLemmatizer()
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)
nltk.download('punkt')




[nltk_data] Downloading package punkt to /home/sabrine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])



## lemmaztize and lower each word and remove duplicates

In [37]:
nltk.download('wordnet')
nltk.download('omw-1.4')
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print(words)

["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


[nltk_data] Downloading package wordnet to /home/sabrine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sabrine/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## sort classes

In [38]:

classes = sorted(list(set(classes)))

## documents = combination between patterns and intents

In [39]:
print (len(documents), "documents")

47 documents


## classes = intents

In [40]:
print (len(classes), "classes", classes)

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


## words = all words, vocabulary


In [41]:

print (len(words), "unique lemmatized words", words)

88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [42]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [43]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

## shuffle our features and turn into np.array


In [44]:
random.shuffle(training)
training = np.array(training)


/tmp/ipykernel_68901/1154947512.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


## create train and test lists. X - patterns, Y - intents


In [45]:
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


## Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons


In [46]:
# equal to number of intents to predict output intent with softmax

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

## Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model


In [47]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

## fitting and saving the model 

In [48]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1970 - accuracy: 0.1915
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0415 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0304 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 1ms/step - loss: 1.9093 - accuracy: 0.3830
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.8984 - accuracy: 0.3191
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6899 - accuracy: 0.4043
Epoch 7/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5093 - accuracy: 0.5957
Epoch 8/200
10/10 [==============================] - 0s 2ms/step - loss: 1.5514 - accuracy: 0.4681
Epoch 9/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3462 - accuracy: 0.4894
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.0681 - accuracy: 0.8085
Epoch 11/